# Rendering
In this tutorial, we'll explain the ins and outs of rendering: the process of creating an image from 3D data.

## 4D-to-video

In Medusa, we try to make it as easy as possible to visualize 4D reconstruction data. As you might have seen in the [quickstart](../getting_started/quickstart), you can use a `VideoRenderer` object for this. Note that this renderer is only available if you have [pytorch3d](https://pytorch3d.org/) installed, which is unfortunately not possible on Windows at the moment.

In [ ]:
from medusa.render import VideoRenderer

The class constructor takes three arguments &mdash; `render_cls`, `shading`, and `lights`. We can ignore the `render_cls` argument (as there is only one render class supported, i.e., `PytorchRenderer`) and the `lights` argument, which we'll discuss later. 

The `shading` argument can be either "flat", which creates a faceted look, or "smooth", which creates a smoother surface using [Phong shading](https://en.wikipedia.org/wiki/Phong_shading). We'll use flat shading for now:

In [ ]:
renderer = VideoRenderer(shading='flat')

The renderer expects the 4D reconstruction data to be wrapped in a `Data4D` object (see [data representation tutorial](./data_representation)). Let's load in the 4D reconstruction (by the 'emoca-coarse' model) from our default video:

In [ ]:
from medusa.data import get_example_h5
data_4d = get_example_h5(load=True, model='emoca-coarse')

To render the 4D data to a video, you use the `VideoRenderer` object's `__call__` method (i.e., calling `renderer()`). This method has two mandatory arguments:

* `f_out`: path where the video will be saved
* `data`: the `Data4D` object

Additionally, this method accepts two optional arguments:

* `video`: path to original video
* `overlay`: colors to project onto the vertices before rendering

If you set the `video` argument to the original video associated with the 4D reconstruction, then the video will be used as a background on which the 4D data is rendered. The default value is `None` (in case the data will be rendered on a white background). 

Let's render our data on top of the video!

In [ ]:
from medusa.data import get_example_video
from IPython.display import Video

vid = get_example_video()
f_out = './flat.mp4'

renderer(f_out, data_4d, video=vid)

# Show result
#Video(f_out, embed=True)

One way to make the visualization a little nicer is by only rendering the face (rather than the full head). To do so, you can use the `apply_vertex_mask` method from the `Data4D` object with the `name` argument set to `'face'`:

In [ ]:
data_4d.apply_vertex_mask('face')

This method basically removes all non-face vertices from the mesh, leaving us with 1787 vertices (instead of the original 5023):

In [ ]:
tuple(data_4d.v.shape)

Now, let's re-render the data (which is a lot faster now too, as it has to work with fewer vertices):

In [ ]:
renderer(f_out, data_4d, video=vid)
Video(f_out, embed=True)

### Overlays

In [ ]:
from medusa.render import Overlay

In [ ]:
data_4d.to_local()
dev = data_4d.v - data_4d.v[0, :, :]
overlay = Overlay(dev, colormap='bwr')
colors = overlay.to_array(data_4d.v[0], tris=data_4d.tris)
renderer = VideoRenderer(shading='flat')
renderer(f_out, data_4d, overlay=colors)
Video(f_out, embed=True)

## 3D-to-image